In [ ]:
import onnx
import onnx_tf
from onnx_tf.backend import prepare
import tensorflow as tf
from ultralytics import YOLO
import cv2
import numpy as np

In [ ]:
# Load YOLOv8 model
model = YOLO('try/sub/best.pt')  # Replace with your model path

# Convert to ONNX
success = model.export(format='onnx') 

if not success:
    raise RuntimeError("ONNX export failed")


In [ ]:
# Load ONNX model
onnx_model = onnx.load('best.onnx')  # Replace with your ONNX file path

# Prepare TensorFlow model (includes weights)
tf_rep = prepare(onnx_model)

# Export TensorFlow SavedModel (includes weights)
tf_rep.export_graph("try/tf_model")


In [ ]:
# Load TensorFlow SavedModel (with weights)
tf_model = tf.saved_model.load("try/tf_model")

# Make predictions
image = cv2.imread("data/Jason_Ad_2.jpg")
resized_image = cv2.resize(image, (1024, 1024)) # nathans size
resized_image = np.expand_dims(resized_image, axis=0)
resized_image = tf.convert_to_tensor(resized_image, dtype=tf.float32)
resized_image = tf.transpose(resized_image, perm=[0, 3, 2, 1])
output_dict = tf_model(images = resized_image)

# Process predictions (boxes, scores, classes)
boxes = output_dict['output0']
scores = output_dict['output1']
classes = output_dict['classes']